In [1]:
import logging
import random

import chirho
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import torch
import torch.nn.functional as F
from plotly.subplots import make_subplots
from pyro.infer import MCMC, NUTS, SVI, Predictive, Trace_ELBO
from pyro.infer.autoguide import (AutoDiagonalNormal, AutoMultivariateNormal,
                                  AutoNormal, init_to_mean, init_to_value)
from pyro.nn import PyroModule
from scipy.stats import lognorm

In [2]:
def add_jittered_com(data, jitter=0.00):
    if "c_trust_jittered" not in data.columns:
        data["c_trust_jittered"] = data["c_trust"] + np.random.uniform(-jitter, jitter, len(data))
    return data


impactDF_grid30_1 = pd.read_csv("../data/communicators/communicators_impact/resultsDF_grid30.csv", index_col=0)
impactDF_grid30_2 = pd.read_csv("../data/communicators/communicators_impact/resultsDF_grid30_2.csv", index_col=0)
impactDF_grid30 = pd.concat([impactDF_grid30_1, impactDF_grid30_2], axis=0)
impactDF_grid30 = impactDF_grid30[impactDF_grid30["time to first food"] < 48]
impactDF_grid30 = impactDF_grid30[impactDF_grid30["reward_patch_dim"].isin([1, 2, 4])]
impactDF_grid30["reward_patch_dim"] = impactDF_grid30["reward_patch_dim"].astype("category")
impactDF_grid30 = add_jittered_com(impactDF_grid30, jitter=0.004)

In [3]:
fig_grid30 = px.scatter(
    impactDF_grid30,
    x="c_trust_jittered",
    y="time to first food",
    color="reward_patch_dim",
    template="presentation",
    title="Trust vs time to first food (grid 30)",
    trendline="lowess",
    opacity=0.5,
    labels={"c_trust_jittered": "coefficient of trust", "reward_patch_dim": "patch size"}
)

fig_grid30.update_traces(marker={"size": 7})
fig_grid30.update_layout(
    width=800,
    height=800,
)

fig_grid30.update_xaxes(showgrid=False)
fig_grid30.update_yaxes(showgrid=False)

In [4]:
# this should approximately be a bit above the
# point of slight inflection int he corresponding smoothed lines

threshold_30 = 0.25

restriction30_low = (impactDF_grid30["c_trust"] >= 0.0) & (impactDF_grid30["c_trust"] <= threshold_30)
restriction30_high = impactDF_grid30["c_trust"] > threshold_30
impactDF_grid30_low = impactDF_grid30[restriction30_low].copy()
impactDF_grid30_high = impactDF_grid30[restriction30_high].copy()

In [5]:
# define model and training

def get_samples(data, base_m, base_s, slope_m=0, slope_s=30, n_iters=2000, n_samples=1000):
    trust = torch.tensor(data["c_trust"].values)
    patch = torch.tensor(data["reward_patch_dim"].astype("category").cat.codes.values).long()
    time = torch.tensor(data["time to first food"].values)

    def model(trust, time, patch):
        with pyro.plate("coefs", 3):
            base = pyro.sample("base", dist.Normal(base_m, base_s))
            slope = pyro.sample("slope", dist.Normal(slope_m, slope_s))

        sig = pyro.sample("sig", dist.LogNormal(3, 0.7))

        with pyro.plate("obs", len(time)):
            pyro.sample("time", dist.Normal(base[patch] + slope[patch] * trust, sig), obs=time)

    guide = AutoMultivariateNormal(model, init_loc_fn=init_to_mean)
    svi = SVI(model, guide, optim.Adam({"lr": 0.01}), loss=Trace_ELBO())

    iterations = []
    losses = []

    pyro.clear_param_store()
    num_iters = n_iters
    for i in range(num_iters):
        elbo = svi.step(trust, time, patch)
        iterations.append(i)
        losses.append(elbo)
        if i % 200 == 0:
            logging.info("Elbo loss: {}".format(elbo))

    fig = px.line(x=iterations, y=losses, title="ELBO loss", template="presentation")
    labels={"iterations": "iteration", "losses": "loss"}
    fig.update_xaxes(showgrid=False, title_text=labels["iterations"])
    fig.update_yaxes(showgrid=False, title_text=labels["losses"])
    fig.update_layout(width=700)
    
    fig.show()

    predictive = Predictive(
        model,
        guide=guide,
        num_samples=n_samples,
    )

    sample = {
        k: v.flatten().reshape(n_samples, -1).detach().cpu().numpy()
        for k, v in predictive(trust, time, patch).items()
        if k != "obs"
    }

    return sample

In [6]:
def plot_coefs(sample, group, grid_size=30):

    df = pd.DataFrame(sample['slope'])
    df.columns = ["1", "2", "4"]
    
    prob_sub_0 = df.applymap(lambda x: 1 if x < 0 else 0).sum()

    print(f"posterior probability of negative slope by patch size: \n {prob_sub_0 / len(df)}")
    
    df_long = df.melt(var_name="patch size", value_name="coefficient of trust")

    fig_svi = px.histogram(df_long["coefficient of trust"],
                           color = df_long["patch size"],
                           template="presentation",
                           nbins=90,
                           title = f"Posterior coefficients of trust ({group}, grid {grid_size})",
                           opacity=0.5,
                           labels={"value": "coefficient of trust", "color": "patch_size"},
                           
                        ) 
                        
    
    fig_svi.update_layout(
        xaxis_title="coefficient of trust",
        yaxis_title="",
        xaxis_range=[-20, 5],
        legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.9),
        yaxis=dict(showgrid=False, showticklabels=False),
        width = 700,
    )

    fig_svi.show()

In [7]:
sample30l = get_samples(impactDF_grid30_low, base_m=10, base_s=10, slope_m=0, slope_s=15, n_iters=600, n_samples= 2000)
sample30h = get_samples(impactDF_grid30_high, base_m=10, base_s=10, slope_m=0, slope_s=15, n_iters=600, n_samples= 2000)

In [8]:
# x axes have a fixed range
# for meaningful comparison

plot_coefs(sample30h, ">0.3", 30)

posterior probability of negative slope by patch size: 
 1    1.000
2    0.998
4    0.898
dtype: float64


In [9]:

plot_coefs(sample30l, "<0.3", 30)

posterior probability of negative slope by patch size: 
 1    0.9970
2    0.9565
4    0.9475
dtype: float64


In [10]:
fig_grid30_linear = px.scatter(
    impactDF_grid30,
    x="c_trust_jittered",
    y="time to first food",
    color="reward_patch_dim",
    template="presentation",
    title="Trust vs time to first food (grid 30)",
    opacity=0.5,
    labels={"c_trust_jittered": "coefficient of trust", "reward_patch_dim": "patch size"}
)

fig_grid30_linear.update_traces(marker={"size": 7})
fig_grid30_linear.update_layout(
    width=800,
    height=800,
)

fig_grid30_linear.update_xaxes(showgrid=False)
fig_grid30_linear.update_yaxes(showgrid=False)


base_means_l = sample30l['base'].mean(axis=0)
base_means_h = sample30h['base'].mean(axis=0)

slope_means_l = sample30l['slope'].mean(axis=0)
slope_means_h = sample30h['slope'].mean(axis=0)


trust_range_l = np.linspace(0, .25, 15)
trust_range_h = np.linspace(0.25, .7, 15)


line_colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]

preds_l = []
preds_h = []
for i in range(3):
    local_pred_l = base_means_l[i] + slope_means_l[i] * trust_range_l
    local_pred_h = base_means_h[i] + slope_means_h[i] * trust_range_h
    
    preds_l.append(local_pred_l)
    preds_h.append(local_pred_h)
    
    fig_grid30_linear.add_shape(type="line",
    x0=trust_range_l[0], y0=local_pred_l[0], x1=trust_range_l[14], y1=local_pred_l[14],
    line=dict(
        color=line_colors[i],
        width=3,
    )
    )
    
    fig_grid30_linear.add_shape(type="line",
    x0=trust_range_h[0], y0=local_pred_h[0], x1=trust_range_h[14], y1=local_pred_h[14],
    line=dict(
        color=line_colors[i],
        width=3,
    )
    )

fig_grid30_linear.add_shape(
    type="line",
    x0=0.25,
    x1=0.25,
    y0=min(min(preds_l[0]), min(preds_h[0])),
    y1=max(max(preds_l[2]), max(preds_h[2])),
    line=dict(
        color="black",
        width=3,
        dash="dash", 
    )
)
    
fig_grid30_linear.update_shapes(dict(xref='x', yref='y'))
fig_grid30_linear.show()
